# Pre-Processing

In [3]:
%pip install pdfplumber
%pip install langchain
%pip install langchain_elasticsearch
%pip install langchain_huggingface
%pip install uuid
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

#Get the file path of the pdf in the current directory
def get_pdf_path():
    list_of_files = os.listdir("./data_source/train_source")
    return list_of_files

In [2]:
import re
import pdfplumber

def pdf2text(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    pages = pdf.pages
    text = ""
    for page in pages:
        sub = page.extract_text()
        text += sub

    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)

    # .를 기준으로 문장단위 chunk 생성
    tokenized_sentence = cleaned_text.split(".")

    return tokenized_sentence

In [3]:
from uuid import uuid4
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore

pdf_path = get_pdf_path()

embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-mpnet-base-v2",
    # model_kwargs={'device':'cuda'}
)

client = ElasticsearchStore(
    es_url="https://agi.veritasgarage.com:443", 
    index_name="dacon_nori_index_2", 
    es_user="elastic", 
    es_password="sukbeom", 
    embedding=embeddings)


for path in pdf_path:
    single_pdf = pdf2text("./data_source/train_source/"+path)
    documents = []

    for sen in single_pdf:
        doc = Document(
            page_content=sen,
            metadata={"source": f"{pdf_path}"},
        )
        documents.append(doc)
        
    uuids = [str(uuid4()) for _ in range(len(documents))]
    
    
    client.add_documents(documents=documents, ids=uuids)
    
    

/opt/homebrew/Caskroom/miniconda/base/envs/dacon/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/Caskroom/miniconda/base/envs/dacon/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

In [7]:
query = "민간임대"
result = client.similarity_search(query, top_k=5)

showResults(result)

Total results:  4
page_content=' 공정투명한 국가공무원 채용
①국가공무원 채용시험
일반
인재채용 운영에 대한 응시자 종 1' metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총괄편).pdf', '2024 나라살림 예산개요.pdf', '보건복지부_노인일자리 및 사회활동지원.pdf', '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', '국토교통부_전세임대(융자).pdf', '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf', '국토교통부_소규모주택정비사업.pdf', '고용노동부_청년일자리창출지원.pdf', '월간 나라재정 2023년 12월호.pdf', '보건복지부_생계급여.pdf', '중소벤처기업부_창업사업화지원.pdf', '고용노동부_조기재취업수당.pdf']"}
page_content='
①유도무기사업의 적기 일반 기관대표
유도무기사업 1' metadata={'source': "['재정통계해설.pdf', '1-1 2024 주요 재정통계 1권.pdf', '국토교통부_민간임대(융자).pdf', '고용노동부_내일배움카드(일반).pdf', '2024년도 성과계획서(총ᄀ